Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [2]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n      

Split the Loaded Data into Chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(data)
texts

[Document(page_content='Fixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n                            \n\nDeposits\nCredit Card\nLoans\nInsurance\nInvestment\nRemittance\

Initialize Pinecone

In [4]:
import os
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)

index_name = "main-index"
namespace = "hlb-fixed-deposit-with-conversation-chain"

if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
  
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

d:\Code\langchain-test\venv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


{}

Create Embeddings and Vectors to Insert to Pinecone Database

In [5]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = Pinecone.from_documents(texts, embeddings, index_name=index_name, namespace=namespace)

Create Retriever for Vector Search

In [6]:
retriever = vector.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.9, "k": 15})
retriever

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x000001A6A6054F50>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.9, 'k': 15})

Creating Prompt Template

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: {question}
  By only basing on this context: {context}
  Don't need to refer to chat history as long as you have enough information from the context.
  If the question does not specify the fixed deposit type, consider it the standard fixed deposit.
  If you don't have enought information from the document, don't answer with your general knowledge, just say you don't have that information.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the question, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input question you to do something creative such as writing a song or making a joke.
"""
prompt = PromptTemplate(template=prompt_template, input_variables=['question', 'context'])

Creating Memory

In [18]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm, memory_key="chat_history", output_key="answer", max_token_limit=500, return_messages=True)

Creating Chain

In [19]:
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks import get_openai_callback

chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=retriever,
  memory=memory,
  verbose=True,
  combine_docs_chain_kwargs={"prompt": prompt},
  return_source_documents=True
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain(question)
    print(f'{cb}\n')
    print(f'\nAnswer: {result["answer"]}')
    for source_documents in result["source_documents"]:
      print(f'\nDocument: {source_documents}')


Running the Chain

In [10]:
ask_question("What is fixed deposit?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Junior Fixed Deposit






More Details



An attractive interest FD account for children below the age of 18










Foreign Currency Fixed Deposit






More Details



A convenient way to invest in foreign currency




































Credit Card Name




















Our Artificial Intelligence Advisor is ready to assist you






Chat with us







×









×  
External Link Disclaimer


You are leaving Hong Leong Bank's website as such our Privacy Notice shall cease. We wish to remind you on our terms on the use of links, Disclaimer and Reservation of Intellectual Property Rights.



Continue
Cancel

Fixed Deposit | Hong Leong Bank Malaysia










Fixed Deposit
 
null

In [ ]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")

In [17]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is fixed deposit?
Assistant: A fixed deposit, in the context of Hong Leong Bank (HLB), is a financial product that allows customers to deposit a lump sum of money for a fixed period, earning a higher interest rate than a regular savings account. HLB offers various fixed deposit accounts tailored to different customer needs, including Junior Fixed Deposit for children below the age of 18, offering attractive interest calculated daily and starting from a placement of RM1,000. Other fixed deposit products include the Foreign Currency Fixed Deposit, which allows investment in foreign currency, and the e-Fixed Deposit, which can be placed online via HLB Connect. Fixed deposits are a secure form of investment, and at HLB, they are protected by PIDM up to RM250,0

In [20]:
ask_question("What are the interest rates for Fixed Deposit?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: What are the interest rates for Fixed Deposit?
  By only basing on this context: Terms and Conditions apply.




Member of PIDM. Protected by PIDM up to RM250,000 for each depositor.























Tenure
Interest Rates (p.a.)
1 month
1.85%
2 months
2.25%
3 months
2.25%
4 months
2.30%
5 months
2.30%
6 months
2.30%
7 months
2.35%
8 months
2.35%
9 months
2.35%
10 months
2.35%
11 months
2.35%
12 months
2.35%
13 - 60 months

2.35%


















Important notice
Please click HERE for the exhaustive list of fees and charges.

















English











e-Fixed Deposit Terms and Conditions of Accounts








General Term & Condition of Accounts






















These may interest you





Edit here to enter
          details for prod

In [21]:
ask_question("Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What are the interest rates for Fixed Deposit?
Assistant: The interest rates for Fixed Deposit at HongLeong Bank (HLB) are as follows:

- 1 month: 2.15% p.a.
- 2 months: 2.25% p.a.
- 3 months: 2.25% p.a.
- 4 months: 2.30% p.a.
- 5 months: 2.30% p.a.
- 6 months: 2.30% p.a.
- 7 months: 2.35% p.a.
- 8 months: 2.35% p.a.
- 9 months: 2.35% p.a.
- 10 months: 2.35% p.a.
- 11 months: 2.35% p.a.
- 12 months: 2.35% p.a.
- 13 - 60 months: 2.35% p.a.

Please note that these rates are subject to change and terms and conditions apply. It is advisable to check with the bank directly for the most current rates.
Follow Up Input: Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.
Stand

In [25]:
ask_question("what is the difference between the percentage of interest rates of flexi fixed deposit and e-fixed deposit")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: what is the difference between the percentage of interest rates of flexi fixed deposit and e-fixed deposit
  By only basing on this context: Member of PIDM. Protected by PIDM up to RM250,000 for each depositor.






















Comes with a Flexi One Account so you can have the convenience of chequing and savings at high interest rates.
Auto-sweeping from Flexi FD to Flexi One Account when there are insufficient funds in your Flexi One Account for in-clearing cheques and standing instruction payments. (Auto-sweep is in multiples of RM3,000. A service charge of RM5 is imposed for each auto-sweep)
 Flexibility to make withdrawals at any branch nationwide.
Consolidated monthly statements. No FD receipt issued.
Access via ATM, Internet Banking and Pho